In [6]:
import warnings

import pandas as pd

warnings.filterwarnings('ignore')

import os

path = os.path.join(os.getcwd(), 'Documents\\tese')

# change working directory
os.chdir(path)

In [7]:
# read data
df = pd.read_excel('data/PrecoMerc.xlsx')
df.head()

,Date,Hour,Session,Price - PT [€/MWh],Price - ES [€/MWh]
0,2008-07-01,1,0,70.31,63.11
1,2008-07-01,2,0,64.54,58.55
2,2008-07-01,3,0,64.08,55.85
3,2008-07-01,4,0,63.50,55.00
4,2008-07-01,5,0,63.28,53.96


In [9]:
# parse hour to timedelta
df['Hour'] = pd.to_timedelta(df['Hour'], unit='h')
df['Date'] = pd.to_datetime(df['Date'])
df['time'] = df['Date'] + df['Hour']
df.head()

,Date,Hour,Session,Price - PT [€/MWh],Price - ES [€/MWh],time
0,2008-07-01,0 days 01:00:00,0,70.31,63.11,2008-07-01 01:00:00
1,2008-07-01,0 days 02:00:00,0,64.54,58.55,2008-07-01 02:00:00
2,2008-07-01,0 days 03:00:00,0,64.08,55.85,2008-07-01 03:00:00
3,2008-07-01,0 days 04:00:00,0,63.50,55.00,2008-07-01 04:00:00
4,2008-07-01,0 days 05:00:00,0,63.28,53.96,2008-07-01 05:00:00
